In [1]:
import os
# os.chdir('../')
%pwd

'c:\\Users\\akish\\Food_Delivery\\research'

In [2]:
import pandas as pd
# import geopy as geo
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler # Handliing Feature Scaling
from sklearn.impute import SimpleImputer #Handling missing values
from sklearn.preprocessing import OrdinalEncoder #Ordinal Encoding
##Pipelines
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
import warnings
warnings.filterwarnings('ignore')

In [3]:
# import warnings
# warnings.filterwarnings('ignore')

In [4]:
df = pd.read_csv("https://raw.githubusercontent.com/AKISHPOTHURI/DataScience/main/Dataset/online_order.csv")
# data.head()

In [5]:
df.head()

,ID,Delivery_person_ID,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Order_Date,Time_Orderd,Time_Order_picked,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min)
0,0xcdcd,DEHRES17DEL01,36.0,4.2,30.327968,78.046106,30.397968,78.116106,12-02-2022,21:55,22:10,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46
1,0xd987,KOCRES16DEL01,21.0,4.7,10.003064,76.307589,10.043064,76.347589,13-02-2022,14:55,15:05,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23
2,0x2784,PUNERES13DEL03,23.0,4.7,18.562450,73.916619,18.652450,74.006619,04-03-2022,17:30,17:40,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21
3,0xc8b6,LUDHRES15DEL02,34.0,4.3,30.899584,75.809346,30.919584,75.829346,13-02-2022,09:20,09:30,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20
4,0xdb64,KNPRES14DEL02,24.0,4.7,26.463504,80.372929,26.593504,80.502929,14-02-2022,19:50,20:05,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41


In [6]:
df.isnull().sum()

ID                                0
Delivery_person_ID                0
Delivery_person_Age            1854
Delivery_person_Ratings        1908
Restaurant_latitude               0
Restaurant_longitude              0
Delivery_location_latitude        0
Delivery_location_longitude       0
Order_Date                        0
Time_Orderd                    1731
Time_Order_picked                 0
Weather_conditions              616
Road_traffic_density            601
Vehicle_condition                 0
Type_of_order                     0
Type_of_vehicle                   0
multiple_deliveries             993
Festival                        228
City                           1200
Time_taken (min)                  0
dtype: int64

In [7]:
# data=df.drop(labels=['ID','Delivery_person_ID'], axis=1)
# df = data.loc[data.isnull().sum(1)>=3]
# data = pd.concat([data, df, df]).drop_duplicates(keep=False)
# df = data[data['City'].notna()]

In [8]:
def data_Cleaning(df):
    try:
        data=df.drop(labels=['ID','Delivery_person_ID'], axis=1)
        df = data.loc[data.isnull().sum(1)>=3]
        data = pd.concat([data, df, df]).drop_duplicates(keep=False)
        df = data[data['City'].notna()]
        for i in df.index:
            if df['Time_Orderd'][i] >= '05:00' and df['Time_Orderd'][i] < '10:00':
                df.loc[i,'day_quaters'] = 'morning'
            elif df['Time_Orderd'][i] >= '10:00' and df['Time_Orderd'][i] < '14:00':
                df.loc[i,'day_quaters'] = 'late morning'
            elif df['Time_Orderd'][i] >= '14:00' and df['Time_Orderd'][i] < '19:00':
                df.loc[i,'day_quaters'] = 'afternoon'
            elif df['Time_Orderd'][i] >= '19:00':
                df.loc[i,'day_quaters'] = 'night'
        df=df.drop(labels=['Time_Orderd','Time_Order_picked','Order_Date'], axis=1)
        df['Festival'].fillna("No", inplace = True) # 98% of the column has No as a value 
        df['day_quaters'].fillna("night", inplace = True) # most of the people ordered at night
        df['Delivery_person_Ratings'].fillna(round(np.mean(df['Delivery_person_Ratings']),1), inplace = True)
        df['Delivery_person_Age'].fillna(round(np.mean(df['Delivery_person_Age'])), inplace = True)
        df['multiple_deliveries'].fillna(round(np.mean(df['multiple_deliveries'])), inplace=True)
        df = df.dropna()
        return df
    except Exception as e:
        print("Exception Occured")
df=data_Cleaning(df)
df

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),day_quaters
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46,night
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23,afternoon
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21,afternoon
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20,morning
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41,night
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45579,30.0,4.8,26.902328,75.794257,26.912328,75.804257,Windy,High,1,Meal,motorcycle,0.0,No,Metropolitian,32,late morning
45580,21.0,4.6,0.000000,0.000000,0.070000,0.070000,Windy,Jam,0,Buffet,motorcycle,1.0,No,Metropolitian,36,night
45581,30.0,4.9,13.022394,80.242439,13.052394,80.272439,Cloudy,Low,1,Drinks,scooter,0.0,No,Metropolitian,16,night
45582,20.0,4.7,11.001753,76.986241,11.041753,77.026241,Cloudy,High,0,Snack,motorcycle,1.0,No,Metropolitian,26,late morning


In [9]:
df.columns

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Weather_conditions',
       'Road_traffic_density', 'Vehicle_condition', 'Type_of_order',
       'Type_of_vehicle', 'multiple_deliveries', 'Festival', 'City',
       'Time_taken (min)', 'day_quaters'],
      dtype='object')

In [10]:
# df = df[df['Delivery_person_Age'].notna()]
# df = df[df['Delivery_person_Ratings'].notna()]
# df = df[df['multiple_deliveries'].notna()]
# df = df[df['Festival'].notna()]
df['Festival'].fillna("No", inplace = True) # 98% of the column has No as a value 
df['day_quaters'].fillna("night", inplace = True) # most of the people ordered at night
df['Delivery_person_Ratings'].fillna(round(np.mean(df['Delivery_person_Ratings']),1), inplace = True)
df['Delivery_person_Age'].fillna(round(np.mean(df['Delivery_person_Age'])), inplace = True)
df['multiple_deliveries'].fillna(round(np.mean(df['multiple_deliveries'])), inplace=True)

In [11]:
df.isnull().sum()

Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken (min)               0
day_quaters                    0
dtype: int64

In [12]:
df = df.dropna()

In [ ]:
# data = pd.read_csv("https://raw.githubusercontent.com/AKISHPOTHURI/DataScience/main/Dataset/online_order.csv")

In [12]:
# def data_Cleaning(data):
#     try:
#         df=data.drop(labels=['ID','Delivery_person_ID'], axis=1)
#         df = df.loc[data.isnull().sum(1)>=3]
#         data1 = pd.concat([data, df, df]).drop_duplicates(keep=False)
#         df = data1[data1['City'].notna()]

#         for i in df.index:
#             if df['Time_Orderd'][i] >= '05:00' and df['Time_Orderd'][i] < '10:00':
#                 df.loc[i,'day_quaters'] = 'morning'
#             elif df['Time_Orderd'][i] >= '10:00' and df['Time_Orderd'][i] < '14:00':
#                 df.loc[i,'day_quaters'] = 'late morning'
#             elif df['Time_Orderd'][i] >= '14:00' and df['Time_Orderd'][i] < '19:00':
#                 df.loc[i,'day_quaters'] = 'afternoon'
#             elif df['Time_Orderd'][i] >= '19:00':
#                 df.loc[i,'day_quaters'] = 'night'

#         df['Festival'].fillna("No", inplace = True) # 98% of the column has No as a value 
#         df['Delivery_person_Ratings'].fillna(round(np.mean(df['Delivery_person_Ratings']),1), inplace = True)
#         df['Delivery_person_Age'].fillna(round(np.mean(df['Delivery_person_Age'])), inplace = True)
#         df['multiple_deliveries'].fillna(round(np.mean(df['multiple_deliveries'])), inplace=True)
#         df['day_quaters'].fillna("night", inplace = True) # most of the people ordered at night
#         # df=df.drop(labels=['Time_Orderd','Time_Order_picked','Order_Date'], axis=1)
#         df = df.dropna()
#         return df
#     except Exception as e:
#         print("Exception occured")
# a =data_Cleaning(data)
# a
            

In [13]:
df.isnull().sum()

Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
Time_taken (min)               0
day_quaters                    0
dtype: int64

In [14]:
data.columns

Index(['ID', 'Delivery_person_ID', 'Delivery_person_Age',
       'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Order_Date', 'Time_Orderd',
       'Time_Order_picked', 'Weather_conditions', 'Road_traffic_density',
       'Vehicle_condition', 'Type_of_order', 'Type_of_vehicle',
       'multiple_deliveries', 'Festival', 'City', 'Time_taken (min)'],
      dtype='object')

In [15]:

# Set the earth's radius (in kilometers)
R = 6371
index = []
# Convert degrees to radians
def deg_to_rad(degrees):
    return degrees * (np.pi/180)

# Function to calculate the distance between two points using the haversine formula
def distcalculate(lat1, lon1, lat2, lon2):
    d_lat = deg_to_rad(lat2-lat1)
    d_lon = deg_to_rad(lon2-lon1)
    a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
    return R * c
  
# Calculate the distance between each pair of points
# data1['distance'] = np.nan

for i in range(len(df)):
    if i in df.index:
        df.loc[i, 'distance'] = distcalculate(df.loc[i, 'Restaurant_latitude'], 
                                        df.loc[i, 'Restaurant_longitude'], 
                                        df.loc[i, 'Delivery_location_latitude'], 
                                        df.loc[i, 'Delivery_location_longitude'])
    else:
        index.append(i)



In [13]:
def calc_distance(df):    
    # Set the earth's radius (in kilometers)
    df = df.dropna()
    R = 6371
    index = []
    # Convert degrees to radians
    def deg_to_rad(degrees):
        return degrees * (np.pi/180)

    # Function to calculate the distance between two points using the haversine formula
    def distcalculate(lat1, lon1, lat2, lon2):
        d_lat = deg_to_rad(lat2-lat1)
        d_lon = deg_to_rad(lon2-lon1)
        a = np.sin(d_lat/2)**2 + np.cos(deg_to_rad(lat1)) * np.cos(deg_to_rad(lat2)) * np.sin(d_lon/2)**2
        c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1-a))
        return R * c
    
    # Calculate the distance between each pair of points
    # data1['distance'] = np.nan

    for i in range(len(df)):
        if i in df.index:
            df.loc[i, 'distance'] = distcalculate(df.loc[i, 'Restaurant_latitude'], 
                                            df.loc[i, 'Restaurant_longitude'], 
                                            df.loc[i, 'Delivery_location_latitude'], 
                                            df.loc[i, 'Delivery_location_longitude'])
        else:
            index.append(i)   
    return df
df = calc_distance(df)

In [14]:
# df = df.dropna()
df.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),day_quaters,distance
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46,night,10.280582
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23,afternoon,6.242319
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21,afternoon,13.787860
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20,morning,2.930258
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41,night,19.396618


In [15]:
df.reset_index(drop=True)

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,Time_taken (min),day_quaters,distance
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,46,night,10.280582
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,23,afternoon,6.242319
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,21,afternoon,13.787860
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,20,morning,2.930258
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,41,night,19.396618
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42695,30.0,4.8,26.902328,75.794257,26.912328,75.804257,Windy,High,1,Meal,motorcycle,0.0,No,Metropolitian,32,late morning,NaN
42696,21.0,4.6,0.000000,0.000000,0.070000,0.070000,Windy,Jam,0,Buffet,motorcycle,1.0,No,Metropolitian,36,night,NaN
42697,30.0,4.9,13.022394,80.242439,13.052394,80.272439,Cloudy,Low,1,Drinks,scooter,0.0,No,Metropolitian,16,night,NaN
42698,20.0,4.7,11.001753,76.986241,11.041753,77.026241,Cloudy,High,0,Snack,motorcycle,1.0,No,Metropolitian,26,late morning,NaN


In [19]:
#Independent and dependent features
X = df.drop(labels=['Time_taken (min)'], axis=1)
Y = df[['Time_taken (min)']]

In [20]:
X.head()

,Delivery_person_Age,Delivery_person_Ratings,Restaurant_latitude,Restaurant_longitude,Delivery_location_latitude,Delivery_location_longitude,Weather_conditions,Road_traffic_density,Vehicle_condition,Type_of_order,Type_of_vehicle,multiple_deliveries,Festival,City,day_quaters,distance
0,36.0,4.2,30.327968,78.046106,30.397968,78.116106,Fog,Jam,2,Snack,motorcycle,3.0,No,Metropolitian,night,10.280582
1,21.0,4.7,10.003064,76.307589,10.043064,76.347589,Stormy,High,1,Meal,motorcycle,1.0,No,Metropolitian,afternoon,6.242319
2,23.0,4.7,18.562450,73.916619,18.652450,74.006619,Sandstorms,Medium,1,Drinks,scooter,1.0,No,Metropolitian,afternoon,13.787860
3,34.0,4.3,30.899584,75.809346,30.919584,75.829346,Sandstorms,Low,0,Buffet,motorcycle,0.0,No,Metropolitian,morning,2.930258
4,24.0,4.7,26.463504,80.372929,26.593504,80.502929,Fog,Jam,1,Snack,scooter,1.0,No,Metropolitian,night,19.396618


In [21]:
#Define which columns should be ordinal-encoded and which should be scaled
categorical_cols = X.select_dtypes(include='object').columns
numerical_cols = X.select_dtypes(exclude='object').columns
# categorical_cols = ['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
#        'Type_of_vehicle', 'Festival', 'City', 'day_quaters']
#numerical_cols
# numerical_cols = ['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
#        'Restaurant_longitude', 'Delivery_location_latitude',
#        'Delivery_location_longitude', 'Vehicle_condition',
#        'multiple_deliveries']

In [22]:
categorical_cols

Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City', 'day_quaters'],
      dtype='object')

In [23]:
numerical_cols

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries', 'distance'],
      dtype='object')

In [24]:
df.dtypes

Delivery_person_Age            float64
Delivery_person_Ratings        float64
Restaurant_latitude            float64
Restaurant_longitude           float64
Delivery_location_latitude     float64
Delivery_location_longitude    float64
Weather_conditions              object
Road_traffic_density            object
Vehicle_condition                int64
Type_of_order                   object
Type_of_vehicle                 object
multiple_deliveries            float64
Festival                        object
City                            object
Time_taken (min)                 int64
day_quaters                     object
distance                       float64
dtype: object

In [25]:
categorical_cols

Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City', 'day_quaters'],
      dtype='object')

In [26]:
city_map = ['Metropolitian','Urban','Semi-Urban']
Festival_map = ['Yes','No']
Road_traffic_density_map = ['Jam','High','Medium','Low']
Type_of_order_map = ['Snack','Meal','Drinks','Buffet']
Type_of_vehicle_map = ['motorcycle','scooter','electric_scooter']
Weather_conditions_map = ["Sunny","Stormy","Sandstorms","Windy","Fog","Cloudy"]
day_quaters_map = ['night','afternoon','morning','late morning']

# Weather_conditions_Map=["Sunny","Stormy","Sandstorms","Windy","Fog","Cloudy"]
# Road_Traffic_Map=["Low","Medium","High","Jam"]
# Type_of_vehicle_map=["bicycle","electric_scooter","scooter","motorcycle"]
# Festival_Map=["No","Yes"]
# City_Map=["Urban","Metropolitian","Semi-Urban"]


In [27]:
#Nuerical Pipeline
num_pipeline = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
    ]
)

#Categorical Pipeline
cat_pipeline = Pipeline(
    steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('OrdinalEncoder', OrdinalEncoder(categories=[Weather_conditions_map,Road_traffic_density_map,Type_of_order_map,Type_of_vehicle_map,Festival_map,
                                                  city_map,day_quaters_map])),
    ('scaler', StandardScaler())
    ]
)

In [17]:
from sklearn.model_selection import train_test_split
def modelTraining(df):
    X = df.drop(labels=['Time_taken (min)'], axis=1)
    Y = df[['Time_taken (min)']]
    categorical_cols = X.select_dtypes(include='object').columns
    numerical_cols = X.select_dtypes(exclude='object').columns
    city_map = ['Metropolitian','Urban','Semi-Urban']
    Festival_map = ['Yes','No']
    Road_traffic_density_map = ['Jam','High','Medium','Low']
    Type_of_order_map = ['Snack','Meal','Drinks','Buffet']
    Type_of_vehicle_map = ['motorcycle','scooter','electric_scooter']
    Weather_conditions_map = ["Sunny","Stormy","Sandstorms","Windy","Fog","Cloudy"]
    day_quaters_map = ['night','afternoon','morning','late morning']
    #Nuerical Pipeline
    num_pipeline = Pipeline(
        steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
        ]
    )

    #Categorical Pipeline
    cat_pipeline = Pipeline(
        steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('OrdinalEncoder', OrdinalEncoder(categories=[Weather_conditions_map,Road_traffic_density_map,Type_of_order_map,Type_of_vehicle_map,Festival_map,
                                                    city_map,day_quaters_map])),
        ('scaler', StandardScaler())
        ]
    )
    
    preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
    ])
    
    X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.30,random_state=42)
    X_train= pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
    X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = modelTraining(df) 

In [28]:
numerical_cols

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries', 'distance'],
      dtype='object')

In [29]:
categorical_cols

Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City', 'day_quaters'],
      dtype='object')

In [30]:
preprocessor = ColumnTransformer([
    ('num_pipeline',num_pipeline,numerical_cols),
    ('cat_pipeline',cat_pipeline,categorical_cols)
])

In [31]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries', 'distance'],
      dtype='object')...
                                                                              'Low'],
                                                                             ['Snack',
                                                                              'Meal',
                                                                              'Drinks',
                                                                              'Buffet'],
                                                                             ['motorcycle',
                                                                              'scooter',
                                                                              'electric_scooter'],
                                                                             ['Yes',
                                                                              'No'],
                                                                             ['Metropolitian',
                                                                              'Urban',
                                                                              'Semi-Urban'],
                                                                             ['night',
                                                                              'afternoon',
                                                                              'morning',
                                                                              'late '
                                                                              'morning']])),
                                                 ('scaler', StandardScaler())]),
                                 Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City', 'day_quaters'],
      dtype='object'))])

In [32]:
#Train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size=0.30,random_state=42)

In [33]:
X_train.shape

(27988, 16)

In [34]:
X_test.shape

(11996, 16)

In [35]:
X_train= pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test = pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [36]:
X_train.head()

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__Restaurant_latitude,num_pipeline__Restaurant_longitude,num_pipeline__Delivery_location_latitude,num_pipeline__Delivery_location_longitude,num_pipeline__Vehicle_condition,num_pipeline__multiple_deliveries,num_pipeline__distance,cat_pipeline__Weather_conditions,cat_pipeline__Road_traffic_density,cat_pipeline__Type_of_order,cat_pipeline__Type_of_vehicle,cat_pipeline__Festival,cat_pipeline__City,cat_pipeline__day_quaters
0,0.077266,-0.107503,1.257218,0.240368,1.301252,0.241124,0.003281,0.435052,-0.051762,0.280088,0.320870,-1.337630,-0.776335,0.138799,1.761221,0.239617
1,1.466159,-1.059958,0.170234,0.147995,0.151960,0.146398,0.003281,0.435052,-0.075244,0.280088,0.320870,0.453042,0.783244,0.138799,1.761221,0.239617
2,-1.485240,1.162436,0.170234,0.147995,0.160119,0.149222,0.003281,0.435052,-0.045753,0.866805,0.320870,0.453042,-0.776335,0.138799,-0.546398,0.239617
3,-0.443570,-0.424988,0.245694,0.101701,0.238072,0.102458,1.227759,0.435052,-0.050732,-0.306628,1.122414,0.453042,0.783244,0.138799,1.761221,-0.743294
4,0.945324,0.209981,-2.225910,-3.326165,-2.361109,-3.325842,1.227759,0.435052,-0.054677,-0.893344,-1.282219,-1.337630,0.783244,0.138799,-0.546398,-0.743294


In [37]:
X_test.head()

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__Restaurant_latitude,num_pipeline__Restaurant_longitude,num_pipeline__Delivery_location_latitude,num_pipeline__Delivery_location_longitude,num_pipeline__Vehicle_condition,num_pipeline__multiple_deliveries,num_pipeline__distance,cat_pipeline__Weather_conditions,cat_pipeline__Road_traffic_density,cat_pipeline__Type_of_order,cat_pipeline__Type_of_vehicle,cat_pipeline__Festival,cat_pipeline__City,cat_pipeline__day_quaters
0,1.118936,0.527466,0.175724,0.152045,0.164530,0.152801,0.003281,-1.326302,-0.050672,-0.306628,1.122414,-0.442294,-0.776335,0.138799,-0.546398,-0.743294
1,-1.311628,0.527466,1.256224,0.238449,1.298847,0.238734,0.003281,-1.326302,-0.056539,0.866805,-1.282219,0.453042,0.783244,0.138799,-0.546398,-0.743294
2,1.466159,-0.742473,0.795634,0.688968,0.809296,0.687367,0.003281,0.435052,-0.075464,-0.306628,-1.282219,1.348379,-0.776335,0.138799,1.761221,-0.743294
3,1.292547,-0.424988,0.795812,0.688833,0.817643,0.690055,1.227759,0.435052,-0.046414,-1.480060,0.320870,-1.337630,2.342824,0.138799,-0.546398,0.239617
4,0.945324,-3.599837,0.173289,0.151503,0.161970,0.152259,0.003281,-1.326302,-0.050670,-0.893344,-1.282219,1.348379,0.783244,0.138799,-0.546398,-0.743294


In [38]:
categorical_cols

Index(['Weather_conditions', 'Road_traffic_density', 'Type_of_order',
       'Type_of_vehicle', 'Festival', 'City', 'day_quaters'],
      dtype='object')

In [39]:
numerical_cols

Index(['Delivery_person_Age', 'Delivery_person_Ratings', 'Restaurant_latitude',
       'Restaurant_longitude', 'Delivery_location_latitude',
       'Delivery_location_longitude', 'Vehicle_condition',
       'multiple_deliveries', 'distance'],
      dtype='object')

In [40]:
X.isnull().sum()

Delivery_person_Age            0
Delivery_person_Ratings        0
Restaurant_latitude            0
Restaurant_longitude           0
Delivery_location_latitude     0
Delivery_location_longitude    0
Weather_conditions             0
Road_traffic_density           0
Vehicle_condition              0
Type_of_order                  0
Type_of_vehicle                0
multiple_deliveries            0
Festival                       0
City                           0
day_quaters                    0
distance                       0
dtype: int64

In [41]:
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import r2_score,mean_absolute_error,mean_squared_error

In [42]:
regression = LinearRegression()
regression.fit(X_train,y_train)

LinearRegression()

In [43]:
regression.coef_

array([[ 2.23784356e+00, -2.34938017e+00,  4.65356090e+02,
        -7.70460022e+02, -4.42701354e+02,  7.70471759e+02,
        -1.87719636e+00,  1.73493373e+00,  1.68844049e+02,
         1.94100306e+00, -3.03051264e+00, -2.48843899e-02,
        -2.02683792e-02, -1.37469392e+00, -6.59319807e-01,
         7.09413028e-01]])

In [44]:
regression.intercept_

array([26.43815207])

In [45]:
import numpy as np
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [46]:
## Train multiple models
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
trained_model_list=[]
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=evaluate_model(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('='*35)
    # print('\n')

LinearRegression
Model Training Performance
RMSE: 6.074689156891127
MAE: 4.855747028999056
R2 score 58.098411297286965
Lasso
Model Training Performance
RMSE: 6.773165974567675
MAE: 5.42405933133445
R2 score 47.908626845483596
Ridge
Model Training Performance
RMSE: 6.301607789826385
MAE: 5.032642826180354
R2 score 54.9094940513359
Elasticnet
Model Training Performance
RMSE: 6.866483601058076
MAE: 5.517356387933907
R2 score 46.463355918098785


In [47]:
regression.intercept_

array([26.43815207])

In [48]:
# new input
new_input = [[36.0,3,30.327968,78.046106,30.397968,78.116106,1,1,2,1,1,3.0,2,1,1,10.280582]]

In [49]:
output = model.predict(new_input)
output

c:\Users\akish\Food_Delivery\delivery\Lib\site-packages\scikit_learn-1.5.2-py3.13-win-amd64.egg\sklearn\base.py:493: UserWarning: X does not have valid feature names, but ElasticNet was fitted with feature names
  warnings.warn(


array([63.38100652])

In [50]:
X_train.columns

Index(['num_pipeline__Delivery_person_Age',
       'num_pipeline__Delivery_person_Ratings',
       'num_pipeline__Restaurant_latitude',
       'num_pipeline__Restaurant_longitude',
       'num_pipeline__Delivery_location_latitude',
       'num_pipeline__Delivery_location_longitude',
       'num_pipeline__Vehicle_condition', 'num_pipeline__multiple_deliveries',
       'num_pipeline__distance', 'cat_pipeline__Weather_conditions',
       'cat_pipeline__Road_traffic_density', 'cat_pipeline__Type_of_order',
       'cat_pipeline__Type_of_vehicle', 'cat_pipeline__Festival',
       'cat_pipeline__City', 'cat_pipeline__day_quaters'],
      dtype='object')

In [51]:
X_train.head()

,num_pipeline__Delivery_person_Age,num_pipeline__Delivery_person_Ratings,num_pipeline__Restaurant_latitude,num_pipeline__Restaurant_longitude,num_pipeline__Delivery_location_latitude,num_pipeline__Delivery_location_longitude,num_pipeline__Vehicle_condition,num_pipeline__multiple_deliveries,num_pipeline__distance,cat_pipeline__Weather_conditions,cat_pipeline__Road_traffic_density,cat_pipeline__Type_of_order,cat_pipeline__Type_of_vehicle,cat_pipeline__Festival,cat_pipeline__City,cat_pipeline__day_quaters
0,0.077266,-0.107503,1.257218,0.240368,1.301252,0.241124,0.003281,0.435052,-0.051762,0.280088,0.320870,-1.337630,-0.776335,0.138799,1.761221,0.239617
1,1.466159,-1.059958,0.170234,0.147995,0.151960,0.146398,0.003281,0.435052,-0.075244,0.280088,0.320870,0.453042,0.783244,0.138799,1.761221,0.239617
2,-1.485240,1.162436,0.170234,0.147995,0.160119,0.149222,0.003281,0.435052,-0.045753,0.866805,0.320870,0.453042,-0.776335,0.138799,-0.546398,0.239617
3,-0.443570,-0.424988,0.245694,0.101701,0.238072,0.102458,1.227759,0.435052,-0.050732,-0.306628,1.122414,0.453042,0.783244,0.138799,1.761221,-0.743294
4,0.945324,0.209981,-2.225910,-3.326165,-2.361109,-3.325842,1.227759,0.435052,-0.054677,-0.893344,-1.282219,-1.337630,0.783244,0.138799,-0.546398,-0.743294
